**CHAPTER 4 신경망 학습**

4.1 데이터에서 학습한다!

    *   신경망의 특징은 데이터를 보고 학습할 수 있다는 것
    *   데이터를 학습한다는 것은 가중치 매개변수의 값을 데이터를 보고 자동으로 결정한다는 뜻

1. 데이터 주도 학습

    *   데이터에서 답을 찾고 데이터에서 패턴을 발견하고 데이터로 이야기를 만드는 것은 기계학습
    *   이미지를 있는 그대로 학습하는 것은 신경망

2. 훈련 데이터와 시험 데이터

    *   훈련 데이터로 학습하고 시험 데이터로 평가함 
    *   범용 능력은 아직 보지 못한 데이터로 문제를 풀어내는 능력
    *   한 데이터셋에만 지나치게 최적화된 상태를 오버피팅이라 함

4.2 손실 함수 

    *   최적의 매개변수 값을 탐색할 때 사용 -> 오차제곱합과 교차 엔트로피를 사용

1.오차제곱합 

    *   각 원소의 출력(추정 값)과 정답 레이블(참 값)의 차를 제곱한 후 그 총함을 구함



In [ ]:
y = [0.1, 0.05, 0.6, 0.0, 0.05, 0.1, 0.0, 0.1, 0.0, 0.0]
t = [0, 0, 1, 0, 0, 0, 0, 0, 0, 0]

In [ ]:
def sum_squares_error(y, t):
  return 0.5*np.sum((y-t)**2)

In [ ]:
#정답은 2
t = [0, 0, 1, 0, 0, 0, 0, 0, 0, 0]
#예1: 2일 확률이 가장 높다고 추정함
y = [0.1, 0.05, 0.6, 0.0, 0.05, 0.1, 0.0, 0.1, 0.0, 0.0]
sum_squares_error(np.array(y), np.array(t))

NameError: ignored

In [ ]:
#예2: 7일 확률이 가장 높다고 추정함
y = [0.1, 0.05, 0.1, 0.0, 0.05, 0.1, 0.0, 0.6, 0.0, 0.0]
sum_squares_error(np.array(y), np.array(t))

NameError: ignored

2. 교차 엔트로피 오차 

    *   정답일 때의 출력이 전체 값을 정하게 됨
    *   정답에 해당하는 출력이 커질수록 0에 다가가다가, 출력이 1일때 0이 됨 반대로 정답일 때 출력이 작아질수록 오차가 커짐



In [ ]:
def cross_entropy_error(y, t):
  delta = 1e-7
  return -np.sum(t*np.log(y+delta))

In [ ]:
t = [0, 0, 1, 0, 0, 0, 0, 0, 0, 0]
y = [0.1, 0.05, 0.6, 0.0, 0.05, 0.1, 0.0, 0.1, 0.0, 0.0]
cross_entropy_error(np.array(y), np.array(t))

NameError: ignored

In [ ]:
y = [0.1, 0.05, 0.1, 0.0, 0.05, 0.1, 0.0, 0.6, 0.0, 0.0]
cross_entropy_error(np.array(y), np.array(t))

NameError: ignored

3. 미니배치 학습

    *   훈련 데이터를 사용해 학습한다
    *   훈련 데이터에 대한 손실 함수의 값을 구하고 그 값을 최대한 줄여주는 매개변수를 구해야함 



In [ ]:
import sys, os
sys.path.append(os.pardir)  
import numpy as np
from dataset.mnist import load_mnist

(x_train, t_train), (x_test, t_test) = \
  load_mnist(normalize=True, one_hot_abel=True)

print(x_train.shape) #(60000, 784)
print(t_train.shape) #(60000, 10)

ModuleNotFoundError: ignored

In [ ]:
train_size = x_train.shape[0]
batch_size = 10
batch_mask = np.random.choice(train_size, batch_size)
x_batch = x_train[batch_mask]
t_batch = t_train[batch_mask]

NameError: ignored

4. (배치용) 교차 엔트로피 오차 구현하기

In [ ]:
def cross_entropy_error2(y, t):
  if y.ndim == 1:
    t = t.reshape(1, t.size)
    y = y.reshape(1, y.size)

  batxh_size = y.shape[0]
  return -np.sum(t*np.log(y+1e-7))/batch_size

In [ ]:
def cross_entropy_error2(y, t):
  if y.ndim == 1:
    t = t.reshape(1, t.size)
    y = y.reshape(1, y.size)

  batxh_size = y.shape[0]
  return -np.sum(t*np.log(y[np.arrange(batch_size), t]+1e-7))/batch_size

5. 왜 손실 함수를 설정하는가?


    *   최적의 매개변수를 탐색할 때 손실 함수의 값을 가능한 한 작게하는 매개 변수를 찾음
    *   이때 매개변수의 미분을 계산하고, 그 미분 값을 단서로 매개변수의 값을 서서히 갱신하는 과정 반복
    *   정확도를 지표로 삼아서는 안됨 -> 미분값이 대부분의 장소에서 0이 되어 매개변수를 갱신할 수 없음






4.3 수치미분

미분
미분은 한순간의 변화량을 표시

In [2]:
# 나쁜 구현의 예
def numerical_diff(f, x):
  h=1e-50 
  return (f(x-h)-f(x))/h

In [1]:
def numerical_diff(f,x):
  h=1e-4
  return (f(x+h)-f(x-h))/(2*h)

2.수치 미분의 예

In [ ]:
def function_1(x):
  return 0.01*x**2+0.1*x
import numpy as np
import matplotlib.pylab as plt
x=np.arange(0.0,20.0,0.1)
y=function_1(x)
plt.xlabel("x")
plt.ylabel("f(x)")
plt.plot(x,y)
plt.show()
print(numerical_diff(function_1,5)) 
print(numerical_diff(function_1, 10))

3.편미분

In [ ]:
def function_2(x):
  return x[0]**2+x[1]**2
  #또는 return np.sum(x**2)

In [ ]:
def function_tmp1(x0):
  return x0*x0+4.0**2.0
numerical_diff(function_tmp1,3.0)

In [ ]:
def function_tmp2(x1):
  return 3.0**2.0+x1*x1 
  
numerical_diff(function_tmp2,4.0)

4.4 기울기

In [ ]:
def numerical_gradient(f,x): #넘파이 배열의 각 원소에 대해 수치 미분을 구함
  h=1e-4 #0.0001
  grad=np.zeros_like(x) # x와 형상이 같은 배열을 생성
  for idx in range(x.size):
    tmp_val=x[idx]
​
    # f(x+h) 계산
    x[idx]=tmp_val+h
    fxh1=f(x)
​
    #f(x-h) 계산
    x[idx]=tmp_val-h
    fxh2=f(x)
​
    grad[idx]=(fxh1-fxh2)/(2*h)
    x[idx]=tmp_val # 값복원
  return grad

기울기가 가리키는 쪽은 각 장소에서 함수의 출력 값을 가장 크게 줄이는 방향이다.

1.경사법

기울기를 잘 이용해 함수의 최솟값을 찾으려는 것이 경사법

각 지점에서 함수의 값을 낮추는 방안을 제시하는 지표가 기울기라는 것
그러나 기울기가 가리키는 곳에 정말 함수의 최솟값이 있는지, 즉 그쪽이 정말로 나아갈 방향인지는 보장 불가

실제로 복잡한 함수에서는 기울기가 가리키는 방향에 최솟값이 없는 경우가 대부분

최솟값이 되는 장소를 찾는 문제에서는 기울기 정보를 단서로 나아갈 방향을 제시해야함

경사법은 현 위치에서 기울어진 방향으로 일정 거리만큼 이동 -> 이동한 곳에서도 마찬가지로 기울기를 구하고 또 그 기울어진 방향으로 나아가기를 반복  이렇게 해서 함수의 값을 줄이는 것이 경사법

In [ ]:
def gradient_descent(f,init_x, lr=0.01, step_num=100):
    x=init_x

    for i in range(step_num):
        grad=numerical_gradient(f,x) #함수의 기울기를 구함
        x-=lr*grad
    return x

In [ ]:
def function_2(x):
    return x[0]**2+x[1]**2
init_x=np.array([-3.0, 4.0]) # 초기값
gradient_descent(function_2, init_x=init_x, lr=0.1,step_num=100)

In [ ]:
init_x=np.array([-3.0, 4.0]) 
gradient_descent(function_2, init_x=init_x, lr=10.0,step_num=100)

In [ ]:
init_x=np.array([-3.0, 4.0]) 
gradient_descent(function_2, init_x=init_x, lr=1e-10,step_num=100)

2. 신경망에서의 기울기 

가중치 매개변수에 대한 손실 함수의 기울기이다

In [ ]:
import sys, os
sys.path.append(os.pardir)
import numpy as np
from common.functions import softmax, cross_entropy_error
from commen.gradient import numerical_gradient
class simpleNet:
    def __init__(self):
        self.W = np.random.randn(2,3) # 정규분포로 초기화
    def predict(self,x): 
        return np.dot(x, self.W)
    def loss(self,x,t): 
        z = self.predict(x) 
        y = softmax(z)
        loss = cross_entropy_error(y,t)

        return loss

In [ ]:
net= simpleNet()
print(net.W)

x=np.array([0.6,0.9])
p=net.predict(x)
print(p)

np.argmax(p)

t=np.array([0,0,1])
net.loss(x,t)

In [ ]:
def f(W):
  return net.loss(x,t)
dw=numerical_gradient(f,net.W) 
print(dw)

In [ ]:
f = lamda w: net.loss(x,t)
dw = numerical_gradient(f,net.W)

4.5 학습 알고리즘 구현하기

전제 : 신경망에는 적응 가능한 가중치와 편향이 있고, 이 가중치와 편향을 훈련 데이터에 적응하도록 조정하는 과정을 학습이라함

1단계 : 훈련 데이터 중 일부를 무작위로 가져옵니다. 이렇게 선별한 데이터를 미니배치라 하며 그 미니배치의 손실 함수값을 줄이는 것이 목표
2단계 : 미니배치의 손실 함수 값을 줄이기 위해 각 가중치 매개변수의 기울기를 구하고, 기울기는 손실함수의 값을 가장 작게 하는 방향을 제시
3단계 : 가중치 매개변수를 기울기 방향으로 아주 조금 갱신
4단계 : 1~3단계 반복

1. 2층 신경망 클래스 구현하기

In [ ]:
import sys, os
sys.path.append(os.pardir)
from common.functions import 
from common.gradient import numerical_gradient

class TwoLayerNet:
    def __init__(self, input_size, hidden_size, output_size, weight_init_std=0.01):
        #가중치 초기화
        self.params = {}
        self.params['W1'] = weight_init_std \  np.random.randn(input_size, hidden_size)
        self.params['b1'] = np.zeros(hidden_size)
        self.params['W2'] = weight_init_std \ np.random.randn(hidden_size, output_size)
        self.params['b2'] = np.zeros(output_size)
    def predict(self, x):
        W1, W2 = self.params['W1'], self.params['W2']
        b1, b2 = self.params['b1'], self.params['b2']
        
        a1 = np.dot(x, W1) + b1
        z1 = sigmoid(a1)
        a2 = np.dot(z1, W2) + b2
        y = softmax(a2)
        return y

    # x = 입력데이터 , t = 정답 레이블
    def loss(self, x, t):
        y = self.predict(x)
        
        return cross_entropy_error(y, t)
    def accuracy(self, x, t):
        y = self.predict(x)
        y = np.argmax(y, axis=1)
        t = np.argmax(t, axis=1)
        accuracy = np.sum(y==t)/float(x.shape[0])
        return accuracy
        
    # x = 입력데이터 , t = 정답 레이블
    def numerical_gredient(self, x, t):
        loss_W = lambda W: self.loss(x, t)
        
        grads = {}
        grads = ['W1'] = numerical_gredient(loss_W, self.params['W1'])
        grads = ['b1'] = numerical_gredient(loss_W, self.params['b1'])
        grads = ['W2'] = numerical_gredient(loss_W, self.params['W2'])
        grads = ['b2'] = numerical_gredient(loss_W, self.params['b2'])
        
        return grads

In [ ]:
x = np.random.rand(100, 784)#더미 입력 데이터(100장 분량)
y = net.predict(x)

In [ ]:
x = np.random.rand(100, 784)
t = np.random.rand(100, 10)

grads = net.numerical_gradient(x, t)#기울기 계산

grads['W1'].shape#(784, 100)
grads['b1'].shape#(100,)
grads['W2'].shape#(100, 10)
grads['b1'].shape#(10,)

2. 미니배치 학습 구현하기 

In [ ]:
import numpy as np
from dataset.mnist import load_mnist
from two_layer_net import TwoLayerNet

(x_train, t_train), (x_test, t_test) = \
  load_mnist(normalize = True, one_hot_label =True)

train_loss_list = []

#하이퍼파라미터
iters_num = 10000 # 반복 횟수
train_size = x_train.shape[0]
batch_size = 100 # 미니배치 크기
learning_rate = 0.1
network = TwoLayerNet(input_size=784, hidden_size = 50, output_size=10)

for i in range(iters_num):
    # 미니배치 획득
    batch_mask = np.random.choice(train_size, batch_size)
    x_batch = x_train[batch_mask]
    t_batch = t_train[batch_mask]

  # 기울기 계산
  #grad = network.numerical_gradient(x_batch, t_batch) # 성능 열화판
    grad = network.gradient(x_batch, t_batch) # 성능 개선판

  # 매개변수 갱신 
    for key in ('W1', 'b1', 'W2', 'b2'):
    network.params[key] -= learning_rate * grad[key]

    # 학습 경과 기록
    loss = network.loss(x_batch, t_batch)
    train_loss_list.append(loss)

3. 시험 데이터로 평가하기

In [ ]:
import numpy as np
from dataset.mnist import load_mnist
from two_layer_net import TwoLayerNet

(x_train, t_train), (x_test, t_test) = \
  load_mnist(normalize = True, one_hot_label =True)

train_loss_list = []

network = TwoLayerNet(input_size=784, hidden_size=50, output_size=10)

#하이퍼파라미터
iters_num = 10000 # 반복 횟수
train_size = x_train.shape[0]
batch_size = 100 # 미니배치 크기
learning_rate = 0.1

train_loss_list = []
train_acc_list = []
test_loss_list = []

#1에폭당 반복수(1에폭=학습에서 훈련데이터를 모두 소진했을때의 횟수)
iter_per_epoch = max(train_size / batch_size, 1)

for i in range(iters_num):
    # 미니배치 획득
    batch_mask = np.random.choice(train_size, batch_size)
    x_batch = x_train[batch_mask]
    t_batch = t_train[batch_mask]
    
    # 기울기 계산
    grad = network.numerical_gradient(x_batch, t_batch) # 성능 열화판
    #grad = network.gradient(x_batch, t_batch) # 성능 개선판
    
    # 매개변수 갱신 
    for key in ('W1', 'b1', 'W2', 'b2'):
        network.params[key] -= learning_rate * grad[key]
        
    # 학습 경과 기록
    loss = network.loss(x_batch, t_batch)
    train_loss_list.append(loss)
    
    #1에폭당 정확도 계산
    if i%iter_per_epoch==0:
        train_acc = network.accuracy(x_train, t_train)
        test_acc = network.accuracy(x_test, t_test)
        train_acc_list.appnd(train_acc)
        test_acc_list.append(test_acc)
        print("train acc, test acc |"
              + str(train_acc)+","+str(test_acc))